In [ ]:
!conda install -y xgboost numpy pandas scipy

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn import metrics
import numpy as np
import pandas as pd
import pickle
from sklearn.datasets import fetch_openml

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# import the mnist dataset
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

In [ ]:
# separate features and target
X, y = mnist["data"], mnist["target"]
print(X.shape)
print(y.shape)

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                       test_size=0.1, 
                                       random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(y_train.shape)
print(y_test.shape)

## Data Augmentation

In [ ]:
from scipy.ndimage import shift
X_aug_down = shift(np.array(X_train).reshape(63000,28,28), [0,1,0], cval=0)
X_aug_down.shape

In [ ]:
X_aug_up = shift(np.array(X_train).reshape(63000,28,28), [0,-1,0], cval=0)
X_aug_up.shape

In [ ]:
X_aug_right = shift(np.array(X_train).reshape(63000,28,28), [0,0,1], cval=0)
X_aug_right.shape

In [ ]:
X_aug_left = shift(np.array(X_train).reshape(63000,28,28), [0,0,-1], cval=0)
X_aug_left.shape

In [ ]:
X_temp = np.concatenate((X_aug_down, X_aug_up, X_aug_right, X_aug_left))
X_temp.shape

In [ ]:
# Augmented training dataset generated and stored in X_aug and y_aug.
X_aug = np.concatenate((X_train, X_temp.reshape(252000, 784)))
X_aug.shape

In [ ]:
y_aug = np.concatenate((y_train, y_train, y_train, y_train, y_train))
y_aug.shape

In [ ]:
X_train = X_aug
y_train = y_aug


print(len(X_train))
print(len(y_train))

## Preprocessing

In [ ]:
# standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Random Forest

In [ ]:
# modeling: random forest (arbitrary hyperparameters)
model = RandomForestClassifier()

In [ ]:
# define your parameter grid 
param_grid = {
                'max_depth': [5, 10, 15],
                #'criterion': ['entropy', 'gini'],
                'criterion': ['entropy'],
                'min_samples_leaf': [6, 10, 20],
                'class_weight':['balanced', None],
                'n_estimators': [50, 100, 200]
            }

In [ ]:
# establish cross-validation and gridsearch
rf_grid = GridSearchCV(model, param_grid, cv=10, verbose=1,n_jobs=-1)

In [ ]:
# conduct the search (this cell will take a very long time!)
rf_grid.fit(X_train_scaled, y_train)

In [ ]:
# see the best parameters and their score
print("Best parameters:")
print(rf_grid.best_params_)
print("Best score in grid search:")
print(rf_grid.best_score_)
print("best model from grid search:")
print(rf_grid.score(X_test_scaled, y_test))

In [ ]:
# predict
y_preds=rf_grid.predict(X_test_scaled)
print(list(y_preds[:10]))
print(list(y_test[:10]))

In [ ]:
# evaluate
print('Accuracy:', metrics.accuracy_score(y_test, y_preds))
print('Precision:', metrics.precision_score(y_test, y_preds,average='macro'))
print('Recall:', metrics.recall_score(y_test, y_preds,average='macro'))
print('F1 Score:', metrics.f1_score(y_test, y_preds,average='macro'))

## Pickle the model

In [ ]:
# random forest
f = open('rf_grid_model.pkl', 'wb')
pickle.dump(rf_grid, f)
f.close() 

## END